In [ ]:
# Moduł do przetwarzania wszystkich obrazów w folderze
import os
import streamlit_test2 as STtest2
import shutil
import ocr
import detect_car as detect

def process_all_images_in_folder(folder_path):
    # Pobierz listę wszystkich plików w folderze
    filenames = os.listdir(folder_path)

    # Ustal ścieżki do folderów docelowych plików odrzuconych
    unreadable_folder = '../RentML/dataset/training/unreadable'
    multi_read_folder = '../RentML/dataset/training/multi_read'
    os.makedirs(unreadable_folder, exist_ok=True)
    os.makedirs(multi_read_folder, exist_ok=True)

    
    # Iteruj przez wszystkie pliki
    for i, filename in enumerate(filenames, start=1):
        # Tworzymy pełną ścieżkę do pliku
        full_path = os.path.join(folder_path, filename)
        
        # Sprawdzamy, czy plik jest obrazem
        if full_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Otwórz obraz
            with open(full_path, 'rb') as f:
                img_cv = STtest2.load_image(f)

            # Przetwarzaj obraz
            #preprocessed_img, mileage = STtest2.process_image(img_cv)
                
            # Wykonaj OCR
            mileage = ocr.mileage_ocr(img_cv)
            
            # Zidentyfikuj samochód
            car_type = detect.identify(full_path)

            # Jeśli mileage jest None, przenieś plik do folderu unreadable
            if mileage is None:
                shutil.move(full_path, os.path.join(unreadable_folder, filename))
                continue

            # Jeśli mileage ma więcej niż jedną wartość, przenieś plik do folderu multi_read
            if isinstance(mileage, list) and len(mileage) > 1:
                shutil.move(full_path, os.path.join(multi_read_folder, filename))
                continue

            # Ekstrakcja i dodanie danych
            STtest2.extract_and_append_data(full_path, mileage, car_type)

            # Wyświetlanie danych
            STtest2.load_and_display_data(full_path)

            # Potwierdzenie wykonania pętli
            print(f'Eroded image: {filename}, {i}/{len(filenames)}')

# Wywołanie funkcji na folderze ze zdjęciami
# Set divided into two folders
process_all_images_in_folder('../RentML/dataset/training/Osobowy')
process_all_images_in_folder('../RentML/dataset/training/Dostawczy')

In [5]:
# Wczytaj dane z pliku mileage.json do DataFrame
import pandas as pd
import json

try:
    with open('mileage.json', 'r') as f:
        data = json.load(f)
        if data:
            df = pd.DataFrame(data)
            column_values = df.iloc[:, 3].tolist()
            for value in column_values:
            #for value in df:
                print(value)
        else:
            print("The 'mileage.json' file is empty.")
except FileNotFoundError:
    print("The 'mileage.json' file does not exist.")
except json.JSONDecodeError:
    print("Error decoding JSON file.")


[250284]
[254410]
[256163]
[257050]


# WIPE JSON FILE - CAREFUL

In [ ]:
# Wipe mileage.json file
with open('mileage.json', 'w') as f:
    pass